### Install PYTHON dependencies

In [ ]:
pip install fhirpy Authlib python-dotenv

In [ ]:
import datetime
import os
import uuid

import requests
from authlib.jose import jwt
from dotenv import load_dotenv

from fhirpy import SyncFHIRClient

### load environment variables

`cp example/.env.tpl example/.env`   
`CLIENT_ID` - your epic app Non-Production Client ID   
`PRIVATE_KEY_PATH` - path to your private .pem file (see [documentation](https://fhir.epic.com/Documentation?docId=oauth2&section=Creating-Key-Pair))

In [ ]:
load_dotenv("./example/.env")

### Generate JWT and get access token

In [ ]:
BASE_URL = "https://fhir.epic.com/interconnect-fhir-oauth"

In [ ]:
jwt_header = {"alg": "RS384", "typ": "JWT"}

In [ ]:
# Token expiration time should not be more then 5 minutes since the current
exp = int((datetime.datetime.now() + datetime.timedelta(minutes=4)).timestamp())

payload = {
    "iss": os.getenv("CLIENT_ID"),
    "sub": os.getenv("CLIENT_ID"),
    "aud": f"{BASE_URL}/oauth2/token",
    "jti": str(uuid.uuid4()),
    "exp": exp,
}

In [ ]:
with open(os.getenv("PRIVATE_KEY_PATH")) as f:
    private_key = f.read()

In [ ]:
encoded_token = jwt.encode(jwt_header, payload, private_key)

In [ ]:
headers = requests.structures.CaseInsensitiveDict(
    [("Content-Type", "application/x-www-form-urlencoded")]
)

In [ ]:
data = {
    "grant_type": "client_credentials",
    "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
    "client_assertion": encoded_token
}

In [ ]:
response = requests.post(
    f"{BASE_URL}/oauth2/token",
    headers=headers,
    data=data,
)

In [ ]:
access_token = response.json()["access_token"]

### Init FHIR Client

In [ ]:
FHIR_BASE_URL = f"{BASE_URL}/api/FHIR/R4"

In [ ]:
client = SyncFHIRClient(
    FHIR_BASE_URL,
    authorization=f"Bearer {access_token}",
)

### Fetch resources

In [ ]:
client.resources("Patient").search(family="Mychart").fetch()

In [ ]:
client.execute("Patient/eJzlzKe3KPzAV5TtkxmNivQ3", method="GET")

In [ ]:
client.execute("Patient?gender=female", method="GET")

In [ ]:
client.resources("Patient").search(address=123).fetch()